In [15]:
import plotly.offline as py
import plotly.graph_objs as graph_objs
import pandas as pd
import geopandas as gpd
import numpy as np
import re

mapbox_access_token = "pk.eyJ1IjoibWlzaGtpY2UiLCJhIjoiY2s5MG94bWRoMDQxdjNmcHI1aWI1YnFkYyJ9.eFsHqEMYY7qxa0Pb9USCtQ"

In [16]:
dfg = gpd.read_file('C:/Users/mskac/machineLearning/GasLeakConEd/data/2010_census_tracks_with_geoid.geojson')
dfg.columns = map(str.lower, dfg.columns)
dfg = dfg.applymap(lambda s:s.lower() if type(s) == str else s)

In [17]:
dfg = dfg[['geoid', 'ntaname', 'boro_name', 'shape_area', 'geometry']]
dfg.head()

,geoid,ntaname,boro_name,shape_area,geometry
0,36047051100,east williamsburg,brooklyn,1711548.77397,"MULTIPOLYGON (((-73.94626 40.70560, -73.94798 ..."
1,36047055300,north side-south side,brooklyn,1681865.77484,"MULTIPOLYGON (((-73.95584 40.71759, -73.95644 ..."
2,36047057500,greenpoint,brooklyn,2014844.59084,"MULTIPOLYGON (((-73.95066 40.72900, -73.95073 ..."
3,36047052700,north side-south side,brooklyn,2045672.64233,"MULTIPOLYGON (((-73.95083 40.70914, -73.95076 ..."
4,36047052900,williamsburg,brooklyn,1882580.923,"MULTIPOLYGON (((-73.95024 40.70547, -73.95190 ..."


In [18]:
dfg.rename(columns={'boro_name': 'boro', 'ntaname': 'nta'}, inplace=True)
dfg.head()

,geoid,nta,boro,shape_area,geometry
0,36047051100,east williamsburg,brooklyn,1711548.77397,"MULTIPOLYGON (((-73.94626 40.70560, -73.94798 ..."
1,36047055300,north side-south side,brooklyn,1681865.77484,"MULTIPOLYGON (((-73.95584 40.71759, -73.95644 ..."
2,36047057500,greenpoint,brooklyn,2014844.59084,"MULTIPOLYGON (((-73.95066 40.72900, -73.95073 ..."
3,36047052700,north side-south side,brooklyn,2045672.64233,"MULTIPOLYGON (((-73.95083 40.70914, -73.95076 ..."
4,36047052900,williamsburg,brooklyn,1882580.923,"MULTIPOLYGON (((-73.95024 40.70547, -73.95190 ..."


In [19]:
full_df = pd.read_csv('C:/Users/mskac/machineLearning/GasLeakConEd/data/data_with_centroids.csv')

In [20]:
full_df.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [24]:
full_df.describe()

,geoid,gasleaks_per_person,crimes_per_person,centerLong,centerLat
count,9.540000e+02,954.000000,954.000000,954.000000,954.000000
mean,3.605836e+10,0.018130,0.019991,-73.867843,40.769477
std,3.109543e+07,0.302819,0.166655,0.066386,0.065500
min,3.600500e+10,0.000180,0.001119,-74.002862,40.584764
25%,3.606101e+10,0.001674,0.005712,-73.912702,40.719181
50%,3.608101e+10,0.003621,0.007918,-73.868882,40.757632
75%,3.608105e+10,0.007409,0.011504,-73.820327,40.827863
max,3.608116e+10,9.285714,4.714286,-73.711998,40.912950


In [30]:
full_df[full_df['gasleaks_per_person']<0.003].count()

geoid                  418
gasleaks_per_person    418
crimes_per_person      418
boro                   418
nta                    418
centerLong             418
centerLat              418
hover                  418
dtype: int64

Add bins column

<h3>Bins for neighborhoods</h3>

In [38]:
binsN_trees = [
    "0-500",
    "501-1000",
    "1001-1500",
    "1501-2000",
    "2001-2500",
    "2501-3000",
    "3001-3500",
    "3501-4000",
    "4001-4500",
    "4501-5000"
]
bins_leaks_per_person = [
    "0-0.0015",
    "0.00151-0.003",
    "0.0031-0.0045",
    "0.00451-0.006",
    "0.0061-0.0075",
    "0.00751-0.009",
    "0.0091-10"
]

In [48]:
#full_dfN['bin']=full_dfN['trees/sq.mile'].apply(lambda x: binsN_trees[0] if int(x) <=500 else binsN_trees[1] if int(x) <=1000 else binsN_trees[2] if x <=1500 else binsN_trees[3] if int(x) <=2000 else binsN_trees[4] if int(x) <=2500 else binsN_trees[5] if int(x) <=3000 else binsN_trees[6] if int(x) <=3500 else binsN_trees[7] if int(x) <=4000 else binsN_trees[8] if int(x) <=4500 else binsN_trees[9] )
full_df['bins_leaks_per_person']=full_df['gasleaks_per_person'].apply(lambda x: bins_leaks_per_person[0] if float(x) <=0.0015 else bins_leaks_per_person[1] if float(x) <=0.003 else bins_leaks_per_person[2] if float(x) <=0.0045 else bins_leaks_per_person[3] if float(x) <=0.006 else bins_leaks_per_person[4] if float(x) <=0.0075 else bins_leaks_per_person[5] if float(x) <=0.009 else bins_leaks_per_person[6] )

In [49]:
dfg['geoid']= dfg['geoid'].astype('int64')

<h3> Create geojson files according to bins</h3>

In [51]:
merged = dfg.merge(full_df, on='geoid' )
for bin in bins_leaks_per_person:
    print(bin)
    geo_layer = merged[(merged['bins_leaks_per_person'] == bin)] # geo
    try:
        geo_layer.to_file("C:/Users/mskac/machineLearning/GasLeakConEd/data/geolayers/gasleaks_per_person/{}.geojson".format(bin), driver='GeoJSON')
    except:
        print('err')

0-0.0015
0.00151-0.003
0.0031-0.0045
0.00451-0.006
0.0061-0.0075
0.00751-0.009
0.0091-10
